# Code-to-Code Comparison: Dinwoodie with all events as 100% operation reduction

National Renewable Energy Laboratory\
Rob Hammond\
16 August 2023

**Note**: As of v0.7, this notebook is no longer advised to be considered due to the more accurate downtime modeling, that render this example invalid. With the full extent of bug fixes in v0.7, the availability in all of the below hover at ~1% because every failure shuts down the turbine/substation.

In [1]:
from copy import deepcopy
from time import perf_counter
from pprint import pprint

import yaml
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from wombat.core import Simulation
from wombat.core.library import DINWOODIE, load_yaml

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

In [2]:
# Converting Labor values to fixed cost input for the base case
tech_salary_annual = 80000
techs = 20
capacity = 240 * 1000  # 240 -> kW
f"{tech_salary_annual * techs / capacity:.4f}"

'6.6667'

In [3]:
configs = [
    "base_100pct_reduction",
    "more_ctvs_100pct_reduction",
    "fewer_ctvs_100pct_reduction",
    "more_techs_100pct_reduction",
    "fewer_techs_100pct_reduction",
    "failure_50_100pct_reduction",
    "failure_200_100pct_reduction",
    "no_hlvs_100pct_reduction",
    "no_weather_100pct_reduction",
    "historic_weather_100pct_reduction",
    "manual_resets_only_100pct_reduction",
    "minor_repairs_only_100pct_reduction",
    "medium_repairs_only_100pct_reduction",
    "major_repairs_only_100pct_reduction",
    "major_replacements_only_100pct_reduction",
    "annual_service_only_100pct_reduction",
]
columns = deepcopy(configs)
results = {
    "availability - time based": [],
    "availability - production based": [],
    "capacity factor - net": [],
    "capacity factor - gross": [],
    "power production": [],
    "task completion rate": [],
    "annual direct O&M cost": [],
    "annual vessel cost": [],
    "ctv cost": [],
    "fsv cost": [],
    "hlv cost": [],
    "annual repair cost": [],
    "annual technician cost": [],
    "ctv utilization": [],
    "fsv utilization": [],
    "hlv utilization": [],
    
}
metrics_dict = {}

In [4]:
for config in configs:
    # Run the simulation
    start = perf_counter()
    sim = Simulation(DINWOODIE, f"{config}.yaml", random_seed=2023)
    sim.run()
    end = perf_counter()
    print(f"{sim.config.name.rjust(50)} | {(end - start) / 60:.2f} m")
    metrics_dict[config] = sim.metrics

                   dinwoodie_base_100pct_reduction | 0.44 m
              dinwoodie_more_ctvs_100pct_reduction | 0.52 m
             dinwoodie_fewer_ctvs_100pct_reduction | 0.51 m
             dinwoodie_more_techs_100pct_reduction | 0.58 m
            dinwoodie_fewer_techs_100pct_reduction | 0.43 m
             dinwoodie_failure_50_100pct_reduction | 0.33 m
            dinwoodie_failure_200_100pct_reduction | 0.53 m
                dinwoodie_no_hlvs_100pct_reduction | 0.34 m
             dinwoodie_no_weather_100pct_reduction | 0.43 m
       dinwoodie_historic_weather_100pct_reduction | 0.37 m
     dinwoodie_manual_resets_only_100pct_reduction | 0.45 m
     dinwoodie_minor_repairs_only_100pct_reduction | 0.34 m
    dinwoodie_medium_repairs_only_100pct_reduction | 0.22 m
     dinwoodie_major_repairs_only_100pct_reduction | 0.21 m
dinwoodie_major_replacements_only_100pct_reduction | 0.20 m
    dinwoodie_annual_service_only_100pct_reduction | 0.43 m


In [5]:
for config, metrics in metrics_dict.items():
    # Gather the results of interest
    years = metrics.events.year.unique().shape[0]
    mil = 1000000
    
    availability_time = metrics.time_based_availability(frequency="project", by="windfarm").values[0][0]
    availability_production = metrics.production_based_availability(frequency="project", by="windfarm").values[0][0]
    cf_net = metrics.capacity_factor(which="net", frequency="project", by="windfarm").values[0][0]
    cf_gross = metrics.capacity_factor(which="gross", frequency="project", by="windfarm").values[0][0]
    power_production = metrics.power_production(frequency="project", by="windfarm").values[0][0]
    completion_rate = metrics.task_completion_rate(which="both", frequency="project").values[0][0]
    parts = metrics.events[["materials_cost"]].sum().sum()
    techs = metrics.project_fixed_costs(frequency="project", resolution="low").operations[0]
    total = metrics.events[["total_cost"]].sum().sum()
    
    equipment = metrics.equipment_costs(frequency="project", by_equipment=True)
    equipment_sum = equipment.sum().sum()
    hlv = equipment[[el for el in equipment.columns if "Heavy Lift Vessel" in el]].sum().sum()
    fsv = equipment[[el for el in equipment.columns if "Field Support Vessel" in el]].sum().sum()
    ctv = equipment[[el for el in equipment.columns if "Crew Transfer Vessel" in el]].sum().sum()
    
    utilization = metrics.service_equipment_utilization(frequency="project")
    hlv_ur = utilization[[el for el in utilization.columns if "Heavy Lift Vessel" in el]].mean().mean()
    fsv_ur = utilization[[el for el in utilization.columns if "Field Support Vessel" in el]].mean().mean()
    ctv_ur = utilization[[el for el in utilization.columns if "Crew Transfer Vessel" in el]].mean().mean()
    
    # Log the results of interest
    results["availability - time based"].append(availability_time)
    results["availability - production based"].append(availability_production)
    results["capacity factor - net"].append(cf_net)
    results["capacity factor - gross"].append(cf_gross)
    results["power production"].append(power_production)
    results["task completion rate"].append(completion_rate)
    results["annual direct O&M cost"].append((total + techs) / mil / years)
    results["annual vessel cost"].append(equipment_sum / mil / years)
    results["ctv cost"].append(ctv / mil / years)
    results["fsv cost"].append(fsv / mil / years)
    results["hlv cost"].append(hlv / mil / years)
    results["annual repair cost"].append(parts / mil / years)
    results["annual technician cost"].append(techs / mil / years)
    results["ctv utilization"].append(ctv_ur)
    results["fsv utilization"].append(fsv_ur)
    results["hlv utilization"].append(hlv_ur)

In [6]:
# Save the results
# pickled dictionary format
with open(DINWOODIE / "results" / "results_dict_100pct_reduction_v0.8.0.yaml", "w") as f:
    yaml.dump(results, f, default_flow_style=False, sort_keys=False)

# dataframe/csv format
results_df = pd.DataFrame(results.values(), columns=columns, index=results.keys()).fillna(0)
results_df.to_csv(DINWOODIE / "results" / "results_data_100pct_reduction_v0.6.0.csv", index_label="result")

In [7]:
results_df

,base_100pct_reduction,more_ctvs_100pct_reduction,fewer_ctvs_100pct_reduction,more_techs_100pct_reduction,fewer_techs_100pct_reduction,failure_50_100pct_reduction,failure_200_100pct_reduction,no_hlvs_100pct_reduction,no_weather_100pct_reduction,historic_weather_100pct_reduction,manual_resets_only_100pct_reduction,minor_repairs_only_100pct_reduction,medium_repairs_only_100pct_reduction,major_repairs_only_100pct_reduction,major_replacements_only_100pct_reduction,annual_service_only_100pct_reduction
availability - time based,0.37,0.29,0.37,0.37,0.37,0.24,0.45,0.66,0.38,0.32,0.02,0.04,0.40,1.00,0.98,0.99
availability - production based,0.37,0.30,0.37,0.37,0.37,0.24,0.45,0.67,0.00,0.32,0.02,0.04,0.39,1.00,0.98,0.99
capacity factor - net,0.18,0.14,0.18,0.18,0.18,0.11,0.22,0.32,0.00,0.16,0.01,0.02,0.19,0.48,0.47,0.47
capacity factor - gross,0.48,0.48,0.48,0.48,0.48,0.48,0.48,0.48,0.00,0.48,0.48,0.48,0.48,0.48,0.48,0.48
power production,"3,723.63","2,974.04","3,723.63","3,723.63","3,723.63","2,394.55","4,548.62","6,682.40",0.00,"2,614.87",160.52,358.10,"3,915.45","10,025.20","9,853.47","9,889.86"
task completion rate,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.97,1.00
annual direct O&M cost,11.90,14.45,11.90,12.70,11.10,9.74,13.88,3.74,11.68,12.36,3.52,3.76,3.96,3.83,8.83,4.85
annual vessel cost,8.23,8.46,8.23,8.23,8.23,6.63,9.29,1.92,8.06,8.42,1.92,1.92,1.92,2.08,6.16,1.92
ctv cost,1.92,3.20,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92
fsv cost,0.13,0.19,0.13,0.13,0.13,0.14,0.22,0.00,0.05,0.17,0.00,0.00,0.00,0.16,0.00,0.00
